In [1]:
import sqlite3
from collections.abc import Collection

In [17]:
conn = sqlite3.connect('example.sqlite')
cursor = conn.cursor()

# Получаем список всех таблиц
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]


for table in tables:
    print(f"Содержимое таблицы {table}:")
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print("-" * 40)  

conn.close()


Содержимое таблицы presidents:
('Yeltsin', 999, 'Russia')
('Trump', 1337, 'US')
('Big Man Tyrone', 101, 'Kekistan')
----------------------------------------
Содержимое таблицы books:
('Farenheit 451', 'Bradbury')
('Brave New World', 'Huxley')
('1984', 'Orwell')
----------------------------------------


In [5]:
class TableData(Collection):
     
    def __init__(self, database_name: str, table_name: str):
        "Инициализация с указанием имени базы данных и таблицы."
        self.database_name = database_name
        self.table_name = table_name
    
    def _connect(self):
        "Создаёт и возвращает соединение с базой данных."
        return sqlite3.connect(self.database_name)
    
    def __len__(self):
        "Возвращает количество записей в таблице."
        with self._connect() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {self.table_name}")
            return cursor.fetchone()[0]
    
    def __getitem__(self, name: str):
        "Возвращает запись из таблицы по значению name."
        with self._connect() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT * FROM {self.table_name} WHERE name = ?", (name,))
            row = cursor.fetchone()
            if row is None:
                raise KeyError(f"No entry found with name: {name}")
            return row
    
    def __contains__(self, name: str):
        "Проверяет, есть ли запись с данным name в таблице."
        with self._connect() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT 1 FROM {self.table_name} WHERE name = ?", (name,))
            return cursor.fetchone() is not None
    
    def __iter__(self):
        "Позволяет итерироваться по записям таблицы построчно."
        with self._connect() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT * FROM {self.table_name}")
            yield from cursor


In [6]:
presidents = TableData(database_name='example.sqlite', table_name='presidents')


In [7]:
print(len(presidents))  


3


In [18]:
print('Yeltsin' in presidents)  # True, если есть такой президент, иначе False


True


In [10]:
for president in presidents:
    print(president)

('Yeltsin', 999, 'Russia')
('Trump', 1337, 'US')
('Big Man Tyrone', 101, 'Kekistan')


In [11]:
print(presidents['Yeltsin'])  # Выведет кортеж с данными о Борисе Ельцине


('Yeltsin', 999, 'Russia')
